# TF IDF

## Requirements 
Installing requirements using pip

In [79]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [80]:
# Imports
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline

from statistics import mean

import numpy as np
import pandas as pd

import _helpers as hp

## Load Data
For chosing which ASR data change the variable by the key in the paths dictionary

In [81]:
# Defining paths for our data. "ASR" means the TDNNF-LFMMI method, and "new_ASR" means the Wave2vec method.
paths = {
    "ground_truth": "snips/merged_GT_data.csv",
    "ASR": "snips/old_ASR_data.csv",
    "new_ASR": "snips/new_ASR_with_labels.csv",
    "new_ASR_Autocorrect": "snips/new_ASR_Autocorrection_with_labels.csv",
}

# Chosing which ASR data we will use
ASR_data = pd.read_csv(paths["new_ASR"])

# Groundtruth data
GT_data = pd.read_csv(paths["ground_truth"])

In [82]:
ASR_data

,Unnamed: 0,Unnamed: 0.1,transcript,user_action,user_action_num
0,0,0,active igtl like an the entire house,SwitchLightOn,1
1,1,1,activate basement lights,SwitchLightOn,1
2,2,2,a djust the bedroom light in tentity of thirty...,SetLightBrightness,4
3,3,3,can you please change the light color to pink,SetLightColor,5
4,4,4,said the rightness to file,SetLightBrightness,4
...,...,...,...,...,...
1655,1655,1655,turn the large meeting room green,SetLightColor,5
1656,1656,1656,turn the laundry room lights to twenty two,SetLightBrightness,4
1657,1657,1657,don't the light intensity to level thirty nine,SetLightBrightness,4
1658,1658,1658,turned the late on,SwitchLightOn,1


### DATA manipulation

In [83]:
# Get ASR data into a numpy word array per sentence plus a numpy label array

XX_ASR = ASR_data["transcript"].apply(lambda x: x.split(" "))
X_ASR = list(XX_ASR)  # numpy word array per transcript


y_num_ASR = np.array(ASR_data["user_action_num"])  # labels

In [84]:
# Get Groundtruth data into a numpy word array per sentence plus a numpy label array

XX_GT = GT_data["transcript"].apply(lambda x: x.split(" "))
X_GT = list(XX_GT)  # numpy word array per transcript

y_num_GT = np.array(GT_data["user_action_num"])  # labels

In [85]:
# concat the 2 arrays into one
X_all = X_ASR + X_GT

## Keyword to user action

Label each sentence in the dataset acording with keywords to user action

## TF IDF vector representation 

### Create TD IDF model based in the dictionary of our dataset 

In [86]:
# Create TD*IDF vector represenation
dct_X = Dictionary(X_all)
corpusBOW = [dct_X.doc2bow(line) for line in X_all]
model = TfidfModel(corpusBOW)


corpusBOW_ASR = [dct_X.doc2bow(line) for line in X_ASR]
X_ASR_vec = model[corpusBOW_ASR]


corpusBOW_GT = [dct_X.doc2bow(line) for line in X_GT]
X_GT_vec = model[corpusBOW_GT]

In [87]:
def Tfidf2np(X, dct):
    """
    Converts gensim format to numpy array
    Input:
    X - TDidfModel vector (N x lenght("sentence"))
    dct - Dictionary object (lenght("unique words"))
    Output:
    X_np - N x length("unique words")
    """
    N_dict = len(dct)
    N_sent = len(X)
    X_np = np.zeros((N_sent, N_dict))
    i = 0
    for _list in X:
        for word in _list:
            X_np[i, word[0]] = word[1]
        i += 1
    return X_np

In [88]:
# Tokenize data
X_ASR_np = Tfidf2np(X_ASR_vec, dct_X)
X_GT_np = Tfidf2np(X_GT_vec, dct_X)

Lets create a function that vectorizes a sentence based in a embeded text model

In [89]:
def get_TFIDF_feature(sent, dct_X, model):
    """return a vector representative of a string"""

    corpusBOW_sent = [dct_X.doc2bow(line) for line in [sent]]

    sent_vec = model[corpusBOW_sent]

    sent_np = Tfidf2np(sent_vec, dct_X)

    return sent_np

# Classifiers

## Train/Test Split

The function beneath provides the features and labels needed for testing. Using the loaded ASR or not (then using ground truth data) is decided by input. As standard we use the ASR dataset.

In [90]:
def get_train_test_data(type_of_dataset="ASR", train_size=0.9):
    """Retrieves the relevant dataset and splits according to parameter"""
    # If ASR, give ASR features and labels
    if type_of_dataset == "ASR":
        train_features, test_features, train_labels, test_labels = train_test_split(
            X_ASR_np, y_num_ASR, train_size=train_size
        )
    # If the dataset is not the ASR data, use the ground truth data
    else:
        train_features, test_features, train_labels, test_labels = train_test_split(
            X_GT_np, y_num_GT, train_size=train_size
        )

    return train_features, test_features, train_labels, test_labels


def run_classifier(
    classifier_pipe, type_of_dataset="ASR", train_size=0.9, number_of_times=100
):
    """For running the classifiers multiple times, and returning mean accuracy score. Wraps around get_train_test_data"""
    mean_score_list = []
    n = number_of_times
    for i in range(n):
        train_features, test_features, train_labels, test_labels = get_train_test_data(
            type_of_dataset="ASR"
        )
        classifier_pipe.fit(train_features, train_labels)

        classifier_pred_labels = classifier_pipe.predict(test_features)  # predictions

        classifier_score = classifier_pipe.score(test_features, test_labels)  # accuracy

        mean_score_list.append(classifier_score)
    mean_score = mean(mean_score_list)
    return mean_score, classifier_pred_labels, classifier_score, test_labels


## Logistic Regression

In [91]:
lgr = LogisticRegression(max_iter=1000)  # Create the classification model

lgr_pipe = make_pipeline(preprocessing.StandardScaler(), lgr)  # Scale feature space

mean_score, lgr_pred_labels, lgr_score, test_labels = run_classifier(
    classifier_pipe=lgr_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.838


### Model Evaluation

In [92]:
print(
    classification_report(
        test_labels,
        lgr_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, lgr_pred_labels))

print("\nACCURACY:", lgr_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.82      0.82      0.82        33
     SwitchLightOn       0.61      0.90      0.73        21
IncreaseBrightness       0.88      0.79      0.83        28
DecreaseBrightness       0.69      0.77      0.73        26
SetLightBrightness       0.93      0.74      0.82        34
     SetLightColor       0.71      0.62      0.67        24

          accuracy                           0.77       166
         macro avg       0.77      0.77      0.77       166
      weighted avg       0.79      0.77      0.77       166

[[27  1  1  3  0  1]
 [ 1 19  1  0  0  0]
 [ 0  3 22  2  1  0]
 [ 2  3  0 20  0  1]
 [ 1  2  0  2 25  4]
 [ 2  3  1  2  1 15]]

ACCURACY: 0.7710843373493976


## Naive Bayes

### Multinomial Naïve Bayes

In [93]:
mnb = MultinomialNB()  # Create NB model

mnb_pipe = make_pipeline(preprocessing.Normalizer(), mnb)  # Scale feature space

mean_score, mnb_pred_labels, mnb_score, test_labels = run_classifier(
    classifier_pipe=mnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.831


#### Model Evaluation

In [94]:
print(
    classification_report(
        test_labels,
        mnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mnb_pred_labels))

print("\nACCURACY:", mnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.92      0.80      0.86        30
     SwitchLightOn       0.79      0.79      0.79        28
IncreaseBrightness       0.84      0.95      0.89        22
DecreaseBrightness       0.87      0.62      0.73        32
SetLightBrightness       0.83      1.00      0.91        30
     SetLightColor       0.68      0.79      0.73        24

          accuracy                           0.82       166
         macro avg       0.82      0.83      0.82       166
      weighted avg       0.83      0.82      0.82       166

[[24  0  1  2  2  1]
 [ 1 22  2  0  0  3]
 [ 1  0 21  0  0  0]
 [ 0  3  0 20  4  5]
 [ 0  0  0  0 30  0]
 [ 0  3  1  1  0 19]]

ACCURACY: 0.8192771084337349


### Bernoulli naïve Bayes

In [95]:
bnb = BernoulliNB()  # Create the classification model

bnb_pipe = make_pipeline(preprocessing.Normalizer(), bnb)  # Scale feature space

mean_score, bnb_pred_labels, bnb_score, test_labels = run_classifier(
    classifier_pipe=bnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.834


#### Model Evaluation

In [96]:
print(
    classification_report(
        test_labels,
        bnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, bnb_pred_labels))

print("\nACCURACY:", bnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       1.00      0.62      0.76        26
     SwitchLightOn       0.92      0.85      0.88        27
IncreaseBrightness       0.69      0.90      0.78        20
DecreaseBrightness       0.81      0.85      0.83        26
SetLightBrightness       0.96      0.73      0.83        33
     SetLightColor       0.70      0.97      0.81        34

          accuracy                           0.82       166
         macro avg       0.85      0.82      0.82       166
      weighted avg       0.85      0.82      0.82       166

[[16  0  5  3  0  2]
 [ 0 23  1  0  1  2]
 [ 0  1 18  0  0  1]
 [ 0  0  1 22  0  3]
 [ 0  1  1  1 24  6]
 [ 0  0  0  1  0 33]]

ACCURACY: 0.8192771084337349


### Gaussian Naive Bayes

In [97]:
gnb = GaussianNB()  # Create the classification model

gnb_pipe = make_pipeline(preprocessing.StandardScaler(), gnb)  # Scale feature space

mean_score, gnb_pred_labels, gnb_score, test_labels = run_classifier(
    classifier_pipe=gnb_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.64


#### Model Evaluation

In [98]:
print(
    classification_report(
        test_labels,
        gnb_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, gnb_pred_labels))

print("\nACCURACY:", gnb_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.77      0.34      0.48        29
     SwitchLightOn       0.43      0.70      0.53        23
IncreaseBrightness       0.54      0.73      0.62        30
DecreaseBrightness       0.63      0.66      0.64        29
SetLightBrightness       0.83      0.67      0.74        30
     SetLightColor       0.95      0.80      0.87        25

          accuracy                           0.64       166
         macro avg       0.69      0.65      0.65       166
      weighted avg       0.70      0.64      0.65       166

[[10  4 10  4  1  0]
 [ 0 16  3  2  2  0]
 [ 1  5 22  1  1  0]
 [ 0  6  4 19  0  0]
 [ 1  2  2  4 20  1]
 [ 1  4  0  0  0 20]]

ACCURACY: 0.6445783132530121


# SVM

In [100]:
svm = SVC(C=2.78)  # Create the classification model

svm_pipe = make_pipeline(preprocessing.Normalizer(), svm)  # Scale feature space

mean_score, svm_pred_labels, svm_score, test_labels = run_classifier(
    classifier_pipe=svm_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.866


### Model Evaluation

In [101]:
print(
    classification_report(
        test_labels,
        svm_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, svm_pred_labels))

print("\nACCURACY:", svm_score)

                    precision    recall  f1-score   support

    SwitchLightOff       0.85      0.96      0.90        23
     SwitchLightOn       0.88      0.81      0.84        26
IncreaseBrightness       0.96      0.74      0.84        31
DecreaseBrightness       0.61      0.95      0.75        20
SetLightBrightness       0.96      0.85      0.90        27
     SetLightColor       0.95      0.90      0.92        39

          accuracy                           0.86       166
         macro avg       0.87      0.87      0.86       166
      weighted avg       0.89      0.86      0.87       166

[[22  0  0  1  0  0]
 [ 0 21  1  3  1  0]
 [ 2  0 23  5  0  1]
 [ 0  1  0 19  0  0]
 [ 2  0  0  1 23  1]
 [ 0  2  0  2  0 35]]

ACCURACY: 0.8614457831325302


# NEURAL NETWORK

In [102]:
mlp = MLPClassifier(
    hidden_layer_sizes=(400, 100), activation="relu", solver="adam", max_iter=5000
)  # Create the classification model

mlp_pipe = make_pipeline(preprocessing.Normalizer(), mlp)  # Scale feature space

mean_score, mlp_pred_labels, mlp_score, test_labels = run_classifier(
    classifier_pipe=mlp_pipe,
    type_of_dataset="ASR",
    train_size=0.9,
    number_of_times=100,
)

print("Average accuracy score =", round(mean_score, 3))

Average accuracy score = 0.846


### Model Evaluation

In [ ]:
print(
    classification_report(
        test_labels,
        mlp_pred_labels,
        target_names=[
            "SwitchLightOff",
            "SwitchLightOn",
            "IncreaseBrightness",
            "DecreaseBrightness",
            "SetLightBrightness",
            "SetLightColor",
        ],
    )
)

print(confusion_matrix(test_labels, mlp_pred_labels))

print("\nACCURACY:", mlp_score)

# Try Yourself

In [ ]:
def user_friendly(sentence, cls):
    """return action from sentence"""

    sent = sentence.split()
    new_sent = []
    for word in sent:
        new_sent.append(hp.autocorrection(word))

    sent_np = get_TFIDF_feature(new_sent, dct_X, model)
    y_pred = cls.predict(sent_np)
    return hp.indx2action(y_pred)

In [ ]:
user_friendly("I want a increase in the lights of my living room", lgr_pipe)